Imports

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
from sklearn.linear_model import LogisticRegression

import cv2
from google.colab.patches import cv2_imshow
import time

Loading the beans dataset into train and test 

In [2]:
image_train, label_train = tfds.as_numpy(tfds.load(
    'beans',
    split='train',
    batch_size=-1,
    as_supervised=True,
))

print("Train data info:",type(image_train), image_train.shape)

image_test, label_test = tfds.as_numpy(tfds.load(
    'beans',
    split='test',
    batch_size=-1,
    as_supervised=True,
))

print("Test data info:",type(image_test), image_test.shape)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/beans/0.1.0.incompleteN3GL0Z/beans-train.tfrecord


  0%|          | 0/1034 [00:00<?, ? examples/s]

...

Shuffling and writing examples to /root/tensorflow_datasets/beans/0.1.0.incompleteN3GL0Z/beans-test.tfrecord


  0%|          | 0/128 [00:00<?, ? examples/s]

Dataset beans downloaded and prepared to /root/tensorflow_datasets/beans/0.1.0. Subsequent calls will reuse this data.
Instructions for updating:
Use `tf.data.Dataset.get_single_element()`.


Instructions for updating:
Use `tf.data.Dataset.get_single_element()`.


Train data info: <class 'numpy.ndarray'> (1034, 500, 500, 3)
Test data info: <class 'numpy.ndarray'> (128, 500, 500, 3)


Preprocessing both train and test data

In [3]:
x_train = []
x_train_normalized = []

for i in image_train:
  x_train.append(i.astype("float32"))
for i in x_train:
  x_train_normalized.append(i/np.max(i))

print(x_train[0].shape, x_train_normalized[0].shape)

x_train_resized = []
for i in x_train_normalized:
  im = cv2.resize(i, dsize=(128, 128), interpolation=cv2.INTER_CUBIC)
  x_train_resized.append(im)

# grayscale pixel as features for training the logistic model 
# x_train_grey = []
# for i in x_train_resized:
#   grayscale = cv2.cvtColor(i, cv2.COLOR_BGR2GRAY)
#   x_train_grey.append(grayscale) #np.dot(i,[0.2989,0.5870,0.1140])

# print(x_train_resized[0].shape, x_train_grey[0].shape)

x_test = []
x_test_normalized = []

for i in image_test:
  x_test.append(i.astype("float32"))
for i in x_test:
  x_test_normalized.append(i/np.max(i))

x_test_resized = []
for i in x_test_normalized:
  im = cv2.resize(i, dsize=(128, 128), interpolation=cv2.INTER_CUBIC)
  x_test_resized.append(im)

# x_test_grey = []
# for i in x_test_resized:
#   grayscale = cv2.cvtColor(i, cv2.COLOR_BGR2GRAY)
#   x_test_grey.append(grayscale) #np.dot(i,[0.2989,0.5870,0.1140])
#   # x_test_grey.append(np.dot(i,[0.2989,0.5870,0.1140]))

(500, 500, 3) (500, 500, 3)


Flattening the data

In [4]:
x_train_flat = []
for i in x_train_resized: #x_train_grey
  x_train_flat.append(i.flatten())

print(x_train_flat[0].shape)

x_test_flat = []
for i in x_test_resized: #x_test_grey
  x_test_flat.append(i.flatten())

(49152,)


In [5]:
x_train = []
y_train = []
x_test = []
y_test = []
  
for i in range(len(label_train)):
  x_train.append(x_train_flat[i])
  y_train.append(label_train[i])

for i in range(len(label_test)):
  x_test.append(x_test_flat[i])
  y_test.append(label_test[i])

Building the logistic model

In [6]:
model = LogisticRegression()

model.fit(x_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


LogisticRegression()

In [7]:
score = model.score(x_test,y_test)

print(score*100,'%')

64.84375 %
